In [39]:
!pip install pandas

In [40]:
!pip install psycopg[binary]

In [41]:
from psycopg import connect
import pandas as pd

In [42]:
conn_config = {
    "dbname": "videogames_db",
    "user": "postgres",
    "password": "admin",
    "host": "localhost",
    "port": "5432"
}

In [43]:
# Load CSVs
games_df = pd.read_csv("../cleaned_data/games_cleaned.csv")
vgsales_df = pd.read_csv("../cleaned_data/vgsales_cleaned.csv")
merged_df = pd.read_csv("../merged_data/merged_data.csv")

# Standardize column names
games_df.columns = [col.lower().replace(" ", "_") for col in games_df.columns]
vgsales_df.columns = [col.lower().replace(" ", "_") for col in vgsales_df.columns]
merged_df.columns = [col.lower().replace(" ", "_") for col in merged_df.columns]

# Confirm
print("Games columns:", games_df.columns.tolist())
print("VGSales columns:", vgsales_df.columns.tolist())
print("Merged columns:", merged_df.columns.tolist())

Games columns: ['title', 'release_date', 'team', 'rating', 'times_listed', 'number_of_reviews', 'genres', 'summary', 'reviews', 'plays', 'playing', 'backlogs', 'wishlist']
VGSales columns: ['rank', 'name', 'platform', 'year', 'genre', 'publisher', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'global_sales']
Merged columns: ['title', 'release_date', 'team', 'rating', 'times_listed', 'number_of_reviews', 'genres', 'summary', 'reviews', 'plays', 'playing', 'backlogs', 'wishlist', 'rank', 'platform', 'year', 'genre', 'publisher', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'global_sales']


In [ ]:
# Clean merged_df for safe insert

# Replace NaNs with None
merged_df = merged_df.where(pd.notnull(merged_df), None)

In [ ]:
#for games n sales
def insert_dataframe_to_table(df, table_name, conn_config):
    with connect(**conn_config) as conn:
        with conn.cursor() as cur:
            # Convert DataFrame rows into list of tuples
            rows = df.where(pd.notnull(df), None).values.tolist()
            # Generate parameter placeholders
            placeholders = ', '.join(['%s'] * len(df.columns))
            columns = ', '.join(df.columns)
            query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
            for row in rows:
                try:
                    cur.execute(query, row)
                except Exception as e:
                    print(f"Error inserting row: {row}\n{e}")
            conn.commit()
    print(f"Data inserted into {table_name} ✅")

In [19]:
insert_dataframe_to_table(games_df, "games", conn_config)

Data inserted into games ✅


In [20]:
insert_dataframe_to_table(vgsales_df, "vgsales", conn_config)

Data inserted into vgsales ✅


In [ ]:
#func for merged_data

def insert_dataframe_to_table(df, table_name, conn_config):
    with connect(**conn_config) as conn:
        with conn.cursor() as cur:
            for row in df.itertuples(index=False, name=None):
                try:
                    # Convert to pure Python types to avoid NAType issues
                    row = tuple(None if pd.isna(x) else x for x in row)
                    placeholders = ', '.join(['%s'] * len(df.columns))
                    columns = ', '.join(df.columns)
                    query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
                    cur.execute(query, row)
                except Exception as e:
                    print(f"Error inserting row: {row}\n{e}")
            conn.commit()
    print(f"Data inserted into {table_name} ✅")

In [46]:
insert_dataframe_to_table(merged_df, "merged_data", conn_config)

Data inserted into merged_data ✅
